In [1]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import sys

sys.path.append("../python")

import utils


/home/musella/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_words = 1600
glove_dim = 25
hash_dim = 5

In [3]:
words_embed = utils.load_glove_embedding(utils.glove_embedding_path(25))
hash_embed = utils.load_glove_embedding('../data/models/hastags/hash_vectors.d'+str(hash_dim)+'.txt')

In [4]:
df = pd.read_hdf("../data/preproc/tweets.hd5")

tk = Tokenizer(num_words=max_words,
               filters="", # already applied
               lower=True,
               split=" ")
tk.fit_on_texts(df["preproc_text"]) 

In [5]:
reload(utils)
words_embed_mtx, words_unk = utils.fill_embedding_matrix(tk.word_index,words_embed,max_words,glove_dim)
hash_embed_mtx, hash_unk = utils.fill_embedding_matrix(tk.word_index,hash_embed,max_words,hash_dim)


In [6]:
unk = { word: tk.word_index[word] for word in set(words_unk).intersection(set(hash_unk)) }
unk

{'<stop>': 2,
 '<trunc>': 5,
 'chatbot': 961,
 'chatbots': 738,
 'convolutional': 689,
 'datasciencectrl': 1099,
 'datax': 1518,
 'daysofmlcode': 1334,
 'deeplearn007': 1563,
 'interpretability': 1112,
 'ipfconline1': 1414,
 'kaggle': 1269,
 'kdnuggets': 1157,
 'kirkdborne': 735,
 'machinelearning': 491,
 'mikequindazzi': 183,
 'neurips': 424,
 'neurips2018': 1560,
 'nodexl': 994,
 'poptimize': 767,
 'pulipaka': 848,
 'pytorch': 1267,
 'spirosmargaris': 1523,
 'tensorflow': 298,
 'v1': 1263,
 'vanloon': 104,
 'variational': 1439,
 'verisk': 1593,
 'whova': 566}

In [7]:
stop_mtx = np.zeros( (max_words+1, 3) )

stop_mtx[unk.pop("<trunc>"),-2] = 1.
stop_mtx[unk.pop("<stop>"),-1] = 1.

stop_mtx[unk.values(),-3] = 1.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
unk_idx = list(unk.values())
unk

In [ ]:
np.isnan(words_embed_mtx).sum()

In [ ]:
np.isnan(hash_embed_mtx).sum()

In [ ]:
# ! mkdir ../data/models/sequences

import pickle
import json

with open("../data/models/sequences/info.json","w+") as out:
    save = dict(max_words=max_words,unk_idx=unk_idx)
    out.write(json.dumps(save))
    
with open('../data/models/sequences/tokenizer.pkl','wb+') as out:
    out.write( pickle.dumps(tk) )
    
np.save('../data/models/sequences/embed_mtx.npy',words_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',hash_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',stop_mtx)

    


In [ ]:
from sklearn.model_selection import train_test_split

df["sequences"] = tk.texts_to_sequences(df["preproc_text"])

df.to_hdf("../data/preproc/sequences.hd5","squences",columns=["sequences"],mode="w")

# index = np.arange(df.index.shape[0]).astype(np.int)
# train_idx, test_idx = train_test_split(index,test_size=0.2,random_state=123456)
# df_train = df.iloc[train_idx]
# df_test = df.iloc[test_idx]
# df_train.to_hdf("../data/preproc/sequences.hd5","train",columns=["sequences"])
# df_test.to_hdf("../data/preproc/sequences.hd5","test",columns=["sequences"])


In [ ]:
# ?tk.texts_to_sequences